# 네이버 지식인 크롤링

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [4]:
chromedriver = '/Users/com/Downloads/chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

In [5]:
base_url = 'https://kin.naver.com/search/list.naver?query='
url = base_url + quote('여친선물')
driver.get(url)

In [6]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [7]:
lis = soup.select('ul.basic1 > li')
len(lis)

10

In [8]:
li = lis[0]
sub_url = li.select_one('a._searchListTitleAnchor')['href']
sub_url

'https://kin.naver.com/qna/detail.naver?d1id=8&dirId=80101&docId=395035694&qb=7Jes7Lmc7ISg66y8&enc=utf8&section=kin&rank=1&search_sort=0&spq=0'

In [9]:
driver.get(sub_url)

In [16]:
count = int(driver.find_element_by_css_selector('._answerCount.num').text)
answer_page = (count-1) // 5
print(count, answer_page)

8 1


In [17]:
for i in range(answer_page):    # range(0)은 for loop 안돔
    driver.find_element_by_id('nextPageButton').click()
    time.sleep(2)

In [18]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [21]:
divs = soup.select('div.answer-content__item._contentWrap._answer')
len(divs)

8

In [23]:
div = divs[0]
ps = div.select('p.se-text-paragraph')
for p in ps:
    print(p.get_text())

혹시 여친분이 생리통이 있으시다면..여친선물로 복부온열기 어떠세요?
저 생리통 너무 심해서 이번에 남자친구가 복부온열기 선물줬는데..
뭔가 감동이더라구요. 절 많이 생각해서 준 선물 같이 느껴지더라구요.
하고있으면 배가 뜨듯하고 진동마사지도 해줘서 생리통 거의 안느껴지고 좋아요!!
​
제가 쓰는건 배뜨끈 복부온열기이예요. 아래에 링크알려드릴게요.
좋은 여친선물이 되셨으면 좋겠네요.


In [24]:
driver.back()

In [29]:
answer_list = []
for li in lis:
    sub_url = li.select_one('a._searchListTitleAnchor')['href']
    driver.get(sub_url)
    time.sleep(2)

    count = int(driver.find_element_by_css_selector('._answerCount.num').text)
    answer_page = (count-1) // 5
    print(count, end=', ')      #
    for i in range(answer_page):
        driver.find_element_by_id('nextPageButton').click()
        time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    divs = soup.select('div.answer-content__item._contentWrap._answer')
    print(len(divs))        #
    for div in divs:
        ps = div.select('p.se-text-paragraph')
        for p in ps:
            text = p.get_text()
            if text == ' ' or text == '\u200b':
                continue
            answer_list.append(text)

    driver.back()
    time.sleep(2)

8, 8
8, 8
20, 20
8, 8
3, 3
8, 8
10, 10
10, 10
13, 13
12, 12


In [30]:
answer_list[100:110]

['3. 데이트',
 '데이트를 준비하는 것도 부담이 되지 않으면서 할 수 있는 좋은 선물이라고 생각해요',
 '연인과 함께하는 특별한 경험은 소중한 추억이 되니까요 ^^',
 '가까이서 할 수 있는 특별한 데이트로는 주로 연극관람을 추천해드리고 있는데요~',
 '영화보다 이색적이기도 하고 그 시간 그 장소에서만 일어나는 공연의 형식이다보니 다들 좋아하시더라고요 ^^',
 '커플분들께 추천드리는 연극은 바로 <한뼘사이> 인데요~',
 '로맨스 코메디 장르라 커플분들끼리 많이들 편하게 보고 가신다고 하더라구요 ㅎㅎ',
 '자세한 사항은 링크를 참고해 주세요~',
 'https://booking.naver.com/booking/12/bizes/69133',
 '4. 커플아이템']

## 1 ~ 10 페이지 데이터 한꺼번에 하기

In [38]:

for page in range(10,11):
    print('page', page)     #
    url = f'{base_url}{quote("여친선물")}&page={page}'
    driver.get(url)
    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    lis = soup.select('ul.basic1 > li')


    for li in lis:
        sub_url = li.select_one('a._searchListTitleAnchor')['href']
        driver.get(sub_url)
        time.sleep(2)

        count = int(driver.find_element_by_css_selector('._answerCount.num').text)
        answer_page = (count-1) // 5
        #print(count, end=', ')      #
        for i in range(answer_page):
            driver.find_element_by_id('nextPageButton').click()
            time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        divs = soup.select('div.answer-content__item._contentWrap._answer')
        #print(len(divs))        #
        for div in divs:
            ps = div.select('p.se-text-paragraph')
            for p in ps:
                text = p.get_text()
                if text == ' ' or text == '\u200b':
                    continue
                answer_list.append(text)

        driver.back()
        time.sleep(2)

page 10


In [39]:
len(answer_list)

4801

In [40]:
with open('data/new_gift.txt', 'w', encoding='utf-8') as fp:
    for answer in answer_list:
        fp.write(answer+'\n')

In [41]:
driver.close()